In [1]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn
import os, pathlib
import kaggle, kagglehub


In [25]:
num_sold_scale_factor = 5939
test_df = pl.read_parquet('test_preprocessed.parquet')
train_df = pl.read_parquet('train_preprocessed.parquet').drop_nulls()

target_df = train_df['num_sold_scaled']
train_df = train_df.drop(['id', 'num_sold_scaled'])

test_df = test_df.drop(['id', 'num_sold_scaled'])

### Defining the regressor model

In [29]:
import sklearn.ensemble

regressor = sklearn.ensemble.HistGradientBoostingRegressor(
    loss='absolute_error',
    learning_rate=0.1,
    max_iter=1000,
    max_leaf_nodes=None,
    max_depth=None,
    max_features=1.0,
    l2_regularization=1,
    warm_start=True,
    scoring='neg_mean_absolute_percentage_error',
    validation_fraction=0.1,
    verbose=1,
    random_state=42
)

### Training the regressor model

In [59]:
regressor.fit(train_df, target_df)

Binning 0.032 GB of training data: 0.112 s
Binning 0.004 GB of validation data: 0.003 s
Fitting gradient boosted rounds:
[143/10000] 1 tree, 7364 leaves, max depth = 35, train score: -0.02614, val score: -0.04850, in 1.291s
Fit 143 trees in 2.681 s, (910391 total leaves)
Time spent computing histograms: 0.165s
Time spent finding best splits:  0.078s
Time spent applying splits:      0.466s
Time spent predicting:           0.010s


In [109]:
model_score = regressor.score(train_df, target_df)
print('Prediction score:', model_score)

In [131]:
num_sold_scale_factor = 5939

pred_num_sold: np.ndarray = (regressor.predict(test_df) * num_sold_scale_factor).astype(int)
np.save('pred_num_sold.npy', pred_num_sold)

In [123]:
predicted_num_solds = pl.Series('pred_num_sold', np.load('pred_num_sold.npy'))

In [137]:
sample_sub = pl.read_parquet('sample_submission.parquet')
print(sample_sub.columns)
print(sample_sub.height, len(predicted_num_solds))

['id', 'num_sold']
98550 98550


In [127]:
sub_df = sample_sub.hstack([predicted_num_solds]).drop('num_sold').rename({'pred_num_sold': 'num_sold'})

In [136]:
sub_df = sub_df.with_columns([
    (pl.when(sub_df['num_sold'] < 4).then(4).otherwise(sub_df['num_sold'])).alias('num_sold')
]).write_csv('submission.csv')

In [130]:
!kaggle competitions submit -c playground-series-s5e1 -f submission.csv -m "Forecasting Sticker Sales Submission"

100%|██████████████████████████████████████| 1.04M/1.04M [00:14<00:00, 74.1kB/s]
Successfully submitted to Forecasting Sticker Sales